# Data preprocessing notebook
* load raw json format
* clean dataset
* output to csv

In [1]:
import json
from tqdm import tqdm
import pandas as pd
import os

import html
import re 
import numpy as np

/var/folders/nq/z_km61px217dlw3_jg7krlvc0000gn/T/ipykernel_65905/3259503239.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
os.path.getsize("Electronics.json")

11508470738

In [3]:
os.path.getsize("meta_Electronics.json")

10995631365

## metadata dataset (product data)

In [4]:
def get_row_counts(file_path):
    with open(file_path, "r") as file:
        row_counts = 0
        for rows in file:
            row_counts += 1
    return row_counts

def load_json_to_df(file_path, partition = 1):
    row_counts = get_row_counts(file_path)
    with open(file_path, "r") as file:
        # start from first partition
        rows_rolling = 0
        json_object = []
        for rows in tqdm(file):
            if rows_rolling > row_counts / partition:
                print(f"Total rows processed {rows_rolling}/{row_counts}")
                break
            else:
                json_obj = json.loads(rows.strip())
                json_object.append(json_obj)
                rows_rolling += 1
                if rows_rolling % 100000 == 0:
                    print(f"Processing row: {rows_rolling}/{row_counts}")
                if rows_rolling == row_counts:
                    print(f"All rows loaded to json objects: {rows_rolling}/{row_counts}")
                    
    print("pushing json objects to pd.df")
    return pd.DataFrame(json_object)

In [6]:
df_product = load_json_to_df(file_path="meta_Electronics.json", partition = 1)

105339it [00:03, 36742.15it/s]

Processing row: 100000/786445


205085it [00:06, 36533.10it/s]

Processing row: 200000/786445


306979it [00:10, 29539.64it/s]

...

Processing row: 600000/786445


703363it [00:29, 26663.08it/s]

Processing row: 700000/786445


786445it [00:40, 19235.35it/s]


All rows loaded to json objects: 786445/786445
pushing json objects to pd.df


In [7]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786445 entries, 0 to 786444
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   category         786445 non-null  object
 1   tech1            786445 non-null  object
 2   description      786445 non-null  object
 3   fit              786445 non-null  object
 4   title            786445 non-null  object
 5   also_buy         786445 non-null  object
 6   tech2            786445 non-null  object
 7   brand            786445 non-null  object
 8   feature          786445 non-null  object
 9   rank             786445 non-null  object
 10  also_view        786445 non-null  object
 11  main_cat         786445 non-null  object
 12  similar_item     786445 non-null  object
 13  date             786445 non-null  object
 14  price            786445 non-null  object
 15  asin             786445 non-null  object
 16  imageURL         786445 non-null  object
 17  imageURLHi

### df_product: data definition and keep/drop rule
* category: product category and sub-catecory information - *REDUNDANT, DROP*
* tech1: primary product specification informtion in HTML format - *REDUNDANT, DROP*
* description: product description - *IMPORTANT, MUST KEEP*
* fit: this column contains wearable electronic product information in HTML format - *REDUNDANT, DROP*
* title: product title - *IMPORTANT, MUST KEEP*
* also_buy: list of asin of product which customer who bought this also bought - *REDUNDANT, CONSIDER DROPPING*
* tech2: secondary product specification informtion in HTML format - *REDUNDANT, DROP*
* brand: brand of product - *IMPORTANT, MUST KEEP*
* feature: product features - *REDUNDANT, CONSIDER DROPPING*
* rank: product rank, need to be clearned as formating is not unified - *IMPORTANT, TO BE CLEANED, MUST KEEP*
* also_view: list of asin of product which customer who viewed this also viewed - *REDUNDANT, DROP*
* main_cat: main product category - *IMPORTANT, MUST KEEP*
* similar_item: similar item, html content - *REDUNDANT, DROP*
* date: product date - *IMPORTANT, MUST KEEP*
* price: price information - *IMPORTANT, MUST KEEP*
* asin: product id - *IMPORTANT, MUST KEEP*
* imageURL: URL of product image - *REDUNDANT, DROP*
* imageURLHighRes: URL of product image - *REDUNDANT, DROP*
* details: - *REDUNDANT, DROP*


In [8]:
columns_to_drop = [
    'category', 'tech1', 'fit', 'also_buy', 'tech2', 
    'feature', 'also_view', 'similar_item', 'imageURL', 
    'imageURLHighRes', 'details'
    ]
# Dropping the redundant columns
df_product = df_product.drop(columns=columns_to_drop)

In [9]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786445 entries, 0 to 786444
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   description  786445 non-null  object
 1   title        786445 non-null  object
 2   brand        786445 non-null  object
 3   rank         786445 non-null  object
 4   main_cat     786445 non-null  object
 5   date         786445 non-null  object
 6   price        786445 non-null  object
 7   asin         786445 non-null  object
dtypes: object(8)
memory usage: 48.0+ MB


## Data Clearning

In [10]:
## Define function for description and value 

def clean_description(description):
    # Check if the input is a list, and convert it to a string if so
    if isinstance(description, list):
        # Join the list elements into a single string separated by spaces
        description = ' '.join(description)
    
    description = html.unescape(description)
    # Remove HTML tags
    description = re.sub('<.*?>', ' ', description)
    # Replace non-alphanumeric characters with spaces
    description = re.sub('[^0-9a-zA-Z]+', ' ', description)
    # Normalise whitespace to single space
    description = re.sub('\s+', ' ', description)
    # Trim leading and trailing spaces
    description = description.strip()
    # Convert to lowercase
    description = description.lower()
    
    return description


def clean_main_cat(main_cat):
    # Convert any HTML entities to their corresponding characters
    main_cat = html.unescape(main_cat)
    # amazon fashion category is compromised contain url so normalise it to just amazon fashion
    main_cat = main_cat.replace('AMAZON FASHION', 'Amazon Fashion')
    
    return main_cat

In [11]:
## Start cleaning

# Removing all $ signs from the 'price' column
df_product['price'] = df_product['price'].str.replace('$', '', regex=False)

# Convert price column to correct dtype to save memory
df_product['price'] = pd.to_numeric(df_product['price'], errors='coerce')

# Clean description and value
df_product['description'] = df_product['description'].apply(clean_description)
df_product['main_cat'] = df_product['main_cat'].apply(clean_main_cat)



In [12]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786445 entries, 0 to 786444
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   description  786445 non-null  object 
 1   title        786445 non-null  object 
 2   brand        786445 non-null  object 
 3   rank         786445 non-null  object 
 4   main_cat     786445 non-null  object 
 5   date         786445 non-null  object 
 6   price        303289 non-null  float64
 7   asin         786445 non-null  object 
dtypes: float64(1), object(7)
memory usage: 48.0+ MB


In [13]:
# drop duplicated rows
df_product = df_product.drop_duplicates(subset=['asin'])


In [14]:
df_product.info()


<class 'pandas.core.frame.DataFrame'>
Index: 756077 entries, 0 to 786444
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   description  756077 non-null  object 
 1   title        756077 non-null  object 
 2   brand        756077 non-null  object 
 3   rank         756077 non-null  object 
 4   main_cat     756077 non-null  object 
 5   date         756077 non-null  object 
 6   price        293928 non-null  float64
 7   asin         756077 non-null  object 
dtypes: float64(1), object(7)
memory usage: 51.9+ MB


Some columns were delibertly left out to maintain the integrity of column asin
* description - cleaned to just alphanumeric text
* title - cleaned
* brand - cleaned
* rank - untouched, contain multiple category rankings
* main_cat - cleaned
* date - untouched, some dates are still compromise
* price - cleaned
* asin - cleaned

In [15]:
# output to CSV
df_product.to_csv('product_cleaned.csv', index=False)

# main dataset

In [5]:
df_main = load_json_to_df(file_path="Electronics.json", partition = 1)

138423it [00:00, 290788.60it/s]

Processing row: 100000/20994353


245800it [00:00, 287487.79it/s]

Processing row: 200000/20994353


348533it [00:01, 282360.77it/s]

...

Processing row: 20800000/20994353


20952701it [10:35, 378412.63it/s]

Processing row: 20900000/20994353


20994353it [10:35, 33033.31it/s] 


All rows loaded to json objects: 20994353/20994353
pushing json objects to pd.df


In [6]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20994353 entries, 0 to 20994352
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   overall         float64
 1   verified        bool   
 2   reviewTime      object 
 3   reviewerID      object 
 4   asin            object 
 5   style           object 
 6   reviewerName    object 
 7   reviewText      object 
 8   summary         object 
 9   unixReviewTime  int64  
 10  vote            object 
 11  image           object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 1.7+ GB


In [7]:
df_main.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"07 17, 2002",A1N070NS9CJQ2I,0060009810,{'Format:': ' Hardcover'},Teri Adams,This was the first time I read Garcia-Aguilera...,Hit The Spot!,1026864000,NaN,NaN
1,5.0,False,"07 6, 2002",A3P0KRKOBQK1KN,0060009810,{'Format:': ' Hardcover'},Willa C.,"As with all of Ms. Garcia-Aguilera's books, I ...",one hot summer is HOT HOT HOT!,1025913600,NaN,NaN
2,5.0,False,"07 3, 2002",A192HO2ICJ75VU,0060009810,{'Format:': ' Hardcover'},Kit,I've not read any of Ms Aguilera's works befor...,One Hot Summer,1025654400,2,NaN
3,4.0,False,"06 30, 2002",A2T278FKFL3BLT,0060009810,{'Format:': ' Hardcover'},Andres,This romance novel is right up there with the ...,I love this book!,1025395200,3,NaN
4,5.0,False,"06 28, 2002",A2ZUXVTW8RXBXW,0060009810,{'Format:': ' Hardcover'},John,Carolina Garcia Aguilera has done it again. S...,One Hot Book,1025222400,NaN,NaN


### df_main: data definition and keep/drop rule
* overall: rating of the product                                               - `*IMPORTANT, MUST KEEP*`
* verified: boolean                                                            - `*IMPORTANT, MUST KEEP*`
* reviewTime: time of the review (raw)                                         - `*IMPORTANT, MUST KEEP*`
* reviewerID: ID of the reviewer, e.g. A2SUAM1J3GNN3B                          - `*REDUNDANT, DROP*`
* asin: ID of the product, e.g. 0000013714                                     - `*IMPORTANT, MUST KEEP*`
* style: a dictionary of the product metadata, e.g., "Format" is "Hardcover"   - `*REDUNDANT, DROP*`
* reviewerName: name of the reviewer                                           - `*REDUNDANT, DROP*`
* reviewText: text of the review                                               - `*IMPORTANT, MUST KEEP*`
* summary: summary of the review                                               - `*DROP*`
* unixReviewTime: time of the review (Unix time format), e.g. 1026864000	   - `*DROP*`
* vote: helpful votes of the review, e.g. NaN or int                           - `*KEEP*`
* image: images that users post after they have received the product           - `*KEEP*`

In [8]:
columns_to_drop_main = [
    'reviewerID', 'style', 'reviewerName', 'summary', 'unixReviewTime'
    ]
# Dropping the redundant columns
df_main = df_main.drop(columns=columns_to_drop_main)
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20994353 entries, 0 to 20994352
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   overall     float64
 1   verified    bool   
 2   reviewTime  object 
 3   asin        object 
 4   reviewText  object 
 5   vote        object 
 6   image       object 
dtypes: bool(1), float64(1), object(5)
memory usage: 981.1+ MB


In [9]:
# convert image url to boolean
df_main['image'] = df_main['image'].notna() & (df_main['image'] != '')
df_main['image'].value_counts()

image
False    20645630
True       348723
Name: count, dtype: int64

In [10]:
# convert data type
df_main['reviewTime'] = pd.to_datetime(df_main['reviewTime'], format='%m %d, %Y')

In [11]:
# drop duplicated rows
df_main = df_main.drop_duplicates()

In [12]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20576637 entries, 0 to 20994352
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   overall     float64       
 1   verified    bool          
 2   reviewTime  datetime64[ns]
 3   asin        object        
 4   reviewText  object        
 5   vote        object        
 6   image       bool          
dtypes: bool(2), datetime64[ns](1), float64(1), object(3)
memory usage: 981.2+ MB


In [13]:
df_main.head()

,overall,verified,reviewTime,asin,reviewText,vote,image
0,5.0,True,2002-07-17,0060009810,This was the first time I read Garcia-Aguilera...,NaN,False
1,5.0,False,2002-07-06,0060009810,"As with all of Ms. Garcia-Aguilera's books, I ...",NaN,False
2,5.0,False,2002-07-03,0060009810,I've not read any of Ms Aguilera's works befor...,2,False
3,4.0,False,2002-06-30,0060009810,This romance novel is right up there with the ...,3,False
4,5.0,False,2002-06-28,0060009810,Carolina Garcia Aguilera has done it again. S...,NaN,False


In [15]:
# output to CSV
df_main.to_csv('review_cleaned.csv', index=False)